# SDMXThon functionalities
## SDMX Message reading

SDMThon includes a simple method to read any sdmx file, regardless of whether it is a data or metadata file.
The method can be called on stored files or urls.


In [1]:
import sdmxthon

message_metadata = sdmxthon.read_sdmx('input_data/bis_otc_outs.xml')
message_data = sdmxthon.read_sdmx('https://stats.bis.org/api/v1/data/BIS,WS_OTC_DERIV2,1.0/all/all?lastNObservations=3&detail=full')

print(message_metadata.type)
print(message_data.type)

MessageTypeEnum.Metadata
MessageTypeEnum.StructureDataSet


The payload of the messages is in an attribute of the message object:

In [2]:
print(message_metadata.payload)
print(message_data.payload)

{'OrganisationSchemes': {'SDMX:AGENCIES(1.0)': <AgencyScheme - SDMX:AGENCIES(1.0)>}, 'Codelists': {'BIS:CL_AVAILABILITY(1.0)': <Codelist - CL_AVAILABILITY>, 'BIS:CL_BIS_IF_REF_AREA(1.0)': <Codelist - CL_BIS_IF_REF_AREA>, 'BIS:CL_BIS_UNIT(1.0)': <Codelist - CL_BIS_UNIT>, 'BIS:CL_COLLECTION(1.0)': <Codelist - CL_COLLECTION>, 'BIS:CL_CONF_STATUS(1.0)': <Codelist - CL_CONF_STATUS>, 'BIS:CL_DECIMALS(1.0)': <Codelist - CL_DECIMALS>, 'BIS:CL_DER_BASIS(1.0)': <Codelist - CL_DER_BASIS>, 'BIS:CL_DER_INSTR(1.0)': <Codelist - CL_DER_INSTR>, 'BIS:CL_EX_METHOD(1.0)': <Codelist - CL_EX_METHOD>, 'BIS:CL_FREQ(1.0)': <Codelist - CL_FREQ>, 'BIS:CL_ISSUE_MAT(1.0)': <Codelist - CL_ISSUE_MAT>, 'BIS:CL_MARKET_RISK(1.0)': <Codelist - CL_MARKET_RISK>, 'BIS:CL_OBS_STATUS(1.0)': <Codelist - CL_OBS_STATUS>, 'BIS:CL_OD_TYPE(1.0)': <Codelist - CL_OD_TYPE>, 'BIS:CL_RATING(1.0)': <Codelist - CL_RATING>, 'BIS:CL_SECTOR_CPY(1.0)': <Codelist - CL_SECTOR_CPY>, 'BIS:CL_SECTOR_UDL(1.0)': <Codelist - CL_SECTOR_UDL>, 'BIS:CL

## SDMX data functionalities

The DataSet object has a series of SDMX-specific attributes. The data in the dataset are stored as a Pandas Dataframe, in the data attribute:

In [3]:
dataset = message_data.content['BIS:WS_OTC_DERIV2(1.0)']
dataset.data

,FREQ,DER_TYPE,DER_INSTR,DER_RISK,DER_REP_CTY,DER_SECTOR_CPY,DER_CPC,DER_SECTOR_UDL,DER_CURR_LEG1,DER_CURR_LEG2,DER_ISSUE_MAT,DER_RATING,DER_EX_METHOD,DER_BASIS,AVAILABILITY,COLLECTION,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_CONF
0,H,T,W,T,5J,N,5J,A,TO1,TO1,A,B,3,A,K,S,2021-S1,7955.000,A,F
1,H,T,W,T,5J,N,5J,A,TO1,TO1,A,B,3,A,K,S,2021-S2,12381.000,A,F
2,H,T,W,T,5J,N,5J,A,TO1,TO1,A,B,3,A,K,S,2022-S1,10363.000,A,F
3,H,A,T,L,5J,A,5J,A,TO1,TO1,A,A,3,A,K,S,2021-S1,284102.446,A,F
4,H,A,T,L,5J,A,5J,A,TO1,TO1,A,A,3,A,K,S,2021-S2,97640.002,A,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14862,H,S,U,T,5J,A,5J,A,TO1,TO1,A,D,3,A,K,E,2021-S2,350594.000,A,F
14863,H,S,U,T,5J,A,5J,A,TO1,TO1,A,D,3,A,K,E,2022-S1,308992.000,A,F
14864,H,A,M,D,5J,B,5J,A,TO1,CHF,A,A,3,C,K,S,2021-S1,4714.702,A,F
14865,H,A,M,D,5J,B,5J,A,TO1,CHF,A,A,3,C,K,S,2021-S2,4619.951,A,F


The generated data frame has one column for each component of the data structure of the dataset.
This dataset does not have any data structure attached to it, since it has not been loaded:

In [4]:
print(dataset.structure)

None


By assigning a structure to the dataset, it is possible to validate the data against its structure:

In [5]:
dataset.structure = message_metadata.content['DataStructures']['BIS:BIS_DER(1.0)']
dataset.semantic_validation()

[]

If the dataset has structural errors, the output of the semantic_validation method is a list of dictionaries with the errors:

In [6]:
dataset.data['DER_BASIS'] = 'error'
dataset.semantic_validation()

[{'Code': 'SS08',
  'ErrorLevel': 'WARNING',
  'Component': 'DER_BASIS',
  'Type': 'Dimension',
  'Rows': None,
  'Message': 'Value error not compliant with maxLength : 1'},
 {'Code': 'SS04',
  'ErrorLevel': 'CRITICAL',
  'Component': 'DER_BASIS',
  'Type': 'Dimension',
  'Rows': None,
  'Message': 'Wrong value error for dimension DER_BASIS'}]

The list of possible errors can be found in this link: 
https://docs.sdmxthon.meaningfuldata.eu/validations.html

From dataset objects it is possible to generate an SDMX v2.1 message, with any of the different modalities:

In [7]:
from sdmxthon.utils.enums import MessageTypeEnum
from datetime import datetime

dataset.to_xml(
    message_type = MessageTypeEnum.StructureDataSet,
    outputPath = 'structure_dataset_example.xml',
    id_ = 'dataset_id',
    test = 'false',
    prepared = datetime.now(),
    sender = 'MeaningfulData',
    receiver = 'open',
    prettyprint=True
)

## SDMX metadata functionalities

SDMXthon provides a simple way to navigate through the metadata. Using the content method on the Message class and the items method on each item Scheme we can access to the inner metadata classes.

In [8]:
concept_scheme = message_metadata.content['Concepts']['BIS:STANDALONE_CONCEPT_SCHEME(1.0)']
concept_scheme.items

{'TIME_FORMAT': <Concept - TIME_FORMAT>,
 'ADJUST_CODED': <Concept - ADJUST_CODED>,
 'AGG_EQUN': <Concept - AGG_EQUN>,
 'AVAILABILITY': <Concept - AVAILABILITY>,
 'BIS_BLOCK': <Concept - BIS_BLOCK>,
 'BIS_DOC_DATE': <Concept - BIS_DOC_DATE>,
 'BIS_SUFFIX': <Concept - BIS_SUFFIX>,
 'BIS_TOPIC': <Concept - BIS_TOPIC>,
 'BIS_UNIT': <Concept - BIS_UNIT>,
 'BORROWERS_CTY': <Concept - BORROWERS_CTY>,
 'BORROWER_NAT': <Concept - BORROWER_NAT>,
 'BREAKS': <Concept - BREAKS>,
 'CBS_BANK_TYPE': <Concept - CBS_BANK_TYPE>,
 'CBS_BASIS': <Concept - CBS_BASIS>,
 'CG_DTYPE': <Concept - CG_DTYPE>,
 'CIBL_CATEGORY': <Concept - CIBL_CATEGORY>,
 'CIBL_TABLE': <Concept - CIBL_TABLE>,
 'CIBL_TYPE': <Concept - CIBL_TYPE>,
 'COLLECTION': <Concept - COLLECTION>,
 'COLLECTION_DETAIL': <Concept - COLLECTION_DETAIL>,
 'COMPILATION': <Concept - COMPILATION>,
 'COMPILING_ORG': <Concept - COMPILING_ORG>,
 'COVERAGE': <Concept - COVERAGE>,
 'COVERED_AREA': <Concept - COVERED_AREA>,
 'CPSS_DATA_TYPE': <Concept - CPSS

Data Structure Definitions can be accessed in a similar way:

In [9]:
dsd = message_metadata.content['DataStructures']['BIS:BIS_DER(1.0)']
dsd.content

{'dimensions': {'FREQ': <Dimension - FREQ>,
  'DER_TYPE': <Dimension - DER_TYPE>,
  'DER_INSTR': <Dimension - DER_INSTR>,
  'DER_RISK': <Dimension - DER_RISK>,
  'DER_REP_CTY': <Dimension - DER_REP_CTY>,
  'DER_SECTOR_CPY': <Dimension - DER_SECTOR_CPY>,
  'DER_CPC': <Dimension - DER_CPC>,
  'DER_SECTOR_UDL': <Dimension - DER_SECTOR_UDL>,
  'DER_CURR_LEG1': <Dimension - DER_CURR_LEG1>,
  'DER_CURR_LEG2': <Dimension - DER_CURR_LEG2>,
  'DER_ISSUE_MAT': <Dimension - DER_ISSUE_MAT>,
  'DER_RATING': <Dimension - DER_RATING>,
  'DER_EX_METHOD': <Dimension - DER_EX_METHOD>,
  'DER_BASIS': <Dimension - DER_BASIS>,
  'TIME_PERIOD': <TimeDimension - TIME_PERIOD>},
 'measure': <PrimaryMeasure - OBS_VALUE>,
 'attributes': {'TIME_FORMAT': <Attribute - TIME_FORMAT>,
  'OBS_STATUS': <Attribute - OBS_STATUS>,
  'AVAILABILITY': <Attribute - AVAILABILITY>,
  'COLLECTION': <Attribute - COLLECTION>,
  'DECIMALS': <Attribute - DECIMALS>,
  'UNIT_MEASURE': <Attribute - UNIT_MEASURE>,
  'UNIT_MULT': <Attribu